In [186]:
from icecream import ic
from ordered_set import OrderedSet
from itertools import product
import itertools as _itertools
import stim
from idtcorev2 import jacobian_coefficient_calc
from pygsti.circuits.circuit import Circuit
from pygsti.baseobjs.label import Label

# Update important parameters here!
VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV

In [187]:
num_qubits = 1
max_weight = 1
term_dict = {("H", "X"): 0.0001}

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [188]:
HS_index_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=1, max_weight=max_weight
)

pauli_node_attributes = list([p for p in HS_index_iterator])
ca_pauli_node_attributes = list(_itertools.combinations(pauli_node_attributes, 2))

def ca_pauli_weight_filter(pauli_pair, max_weight):
    used_indices_1 = set(
        i for i, ltr in enumerate(str(pauli_pair[0])[1:]) if ltr != "_"
    )
    used_indices_2 = set(
        i for i, ltr in enumerate(str(pauli_pair[1])[1:]) if ltr != "_"
    )
    intersect = used_indices_1.intersection(used_indices_2)
    if len(intersect) > 0 and len(intersect) <= max_weight:
        return True

ca_pauli_node_attributes = [
    ppair
    for ppair in ca_pauli_node_attributes
    if ca_pauli_weight_filter(ppair, max_weight)
]

measure_string_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=num_qubits
)
prep_string_iterator = product([1,-1],[p for p in measure_string_iterator])
measure_string_attributes = list([p for p in measure_string_iterator])
prep_string_attributes = list(a*b for a,b in prep_string_iterator)
prep_meas_pair = list(product(prep_string_attributes, measure_string_attributes))

In [189]:
hs_error_gen_classes = "hs"
ca_error_gen_classes = "ca"


hs_experiment = list(
    product(
        hs_error_gen_classes,
        pauli_node_attributes,
        prep_string_attributes,
        measure_string_attributes,
    )
)
ca_experiment = list(
    product(
        ca_error_gen_classes,
        ca_pauli_node_attributes,
        prep_string_attributes,
        measure_string_attributes,
    )
)



import pandas as pd

# df = pd.DataFrame()
jacobian_coef_dict = {"index": OrderedSet(), "columns": OrderedSet()}
data = {}

# These come back as class, index, prep_str, meas_str, observ_str: coef
# I THINK this is right, should double check and write unit tests
for key in hs_experiment + ca_experiment:
    elt = jacobian_coefficient_calc(*key)
    for el in elt:
        if el:
            observable = ",".join(str(s)[:] for s in el[-4:-1])
            egen = ",".join(str(s) for s in el[:-4])
            coef = int(el[-1])
            jacobian_coef_dict["index"].add(observable)
            jacobian_coef_dict["columns"].add(egen)
            if data.get(egen):
                data[egen].append(coef)
            else:
                data[egen] = [coef]

df = pd.DataFrame(data, index= jacobian_coef_dict["index"], columns=jacobian_coef_dict["columns"])

In [190]:
df

,"H,+X","H,+Y","H,+Z","S,+X","S,+Y","S,+Z","C,+X,+Y","C,+X,+Z","C,+Y,+Z"
"+X,+X,+X",0,0,0,0,-2,-2,0,0,0
"+X,+Y,+Y",0,0,2,0,0,0,2,0,0
"+X,+Z,+Z",0,-2,0,0,0,0,0,2,0
"+Y,+X,+X",0,0,-2,0,0,0,2,0,0
"+Y,+Y,+Y",0,0,0,-2,0,-2,0,0,0
"+Y,+Z,+Z",2,0,0,0,0,0,0,0,2
"+Z,+X,+X",0,2,0,0,0,0,0,2,0
"+Z,+Y,+Y",-2,0,0,0,0,0,0,0,2
"+Z,+Z,+Z",0,0,0,-2,-2,0,0,0,0
"-X,+X,+X",0,0,0,0,2,2,0,0,0


In [191]:
import numpy as np
np.set_printoptions(precision=1, linewidth=1000)

In [192]:
import pygsti
from pygsti.extras import idletomography as idt
from pygsti.baseobjs import Label
gates = ["Gi", "Gx", "Gy", "Gcnot"]
max_lengths = [1, 2, 4, 8, 16]
pspec = pygsti.processors.QubitProcessorSpec(
        num_qubits, gates, geometry="line", nonstd_gate_unitaries={(): num_qubits, "Gi": np.eye((2**num_qubits))},
        availability={"Gi": [tuple(i for i in range(num_qubits))]},
    )
mdl_target = pygsti.models.create_crosstalk_free_model(pspec)
paulidicts = idt.determine_paulidicts(mdl_target)
global_idle_string = [Label("Gi", tuple(i for i in range(num_qubits)))]
idle_experiments = idt.make_idle_tomography_list(
        num_qubits, max_lengths, paulidicts, maxweight=1, idle_string=global_idle_string
    )
idle_experiments

[Circuit(Gy:0Gi:0Gy:0Gy:0Gy:0@(0)),
 Circuit(Gy:0Gi:0Gi:0Gy:0Gy:0Gy:0@(0)),
 Circuit(Gy:0Gi:0Gi:0Gi:0Gi:0Gy:0Gy:0Gy:0@(0)),
 Circuit(Gy:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gy:0Gy:0Gy:0@(0)),
 Circuit(Gy:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gy:0Gy:0Gy:0@(0)),
 Circuit(Gy:0Gi:0Gx:0@(0)),
 Circuit(Gy:0Gi:0Gi:0Gx:0@(0)),
 Circuit(Gy:0Gi:0Gi:0Gi:0Gi:0Gx:0@(0)),
 Circuit(Gy:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gx:0@(0)),
 Circuit(Gy:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gx:0@(0)),
 Circuit(Gy:0Gi:0@(0)),
 Circuit(Gy:0Gi:0Gi:0@(0)),
 Circuit(Gy:0Gi:0Gi:0Gi:0Gi:0@(0)),
 Circuit(Gy:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0@(0)),
 Circuit(Gy:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0@(0)),
 Circuit(Gx:0Gx:0Gx:0Gi:0Gy:0Gy:0Gy:0@(0)),
 Circuit(Gx:0Gx:0Gx:0Gi:0Gi:0Gy:0Gy:0Gy:0@(0)),
 Circuit(Gx:0Gx:0Gx:0Gi:0Gi:0Gi:0Gi:0Gy:0Gy:0Gy:0@(0)),
 Circuit(Gx:0Gx:0Gx:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gy:0Gy:0Gy:0@(0)),
 Circuit(Gx:0Gx:0Gx:0Gi:0Gi:

In [193]:
mdl_datagen = pygsti.models.create_crosstalk_free_model(
pspec, lindblad_error_coeffs={"Gi": term_dict},lindblad_parameterization="GLND")

# Error models! Random with right CP constraints from Taxonomy paper
ds = pygsti.data.simulate_data(
    mdl_datagen, idle_experiments, 10000000, seed=8675309, sample_error="none"
)

In [194]:
from idttools import allerrors, all_full_length_observables, alloutcomes
import collections as _collections
from pygsti.circuits.circuit import Circuit as _Circuit
def report_observed_rates(nqubits,
    dataset,
    max_lengths,
    pauli_basis_dicts,
    maxweight=2,
    idle_string=global_idle_string):
    
    all_fidpairs = dict(enumerate(idt.idle_tomography_fidpairs(nqubits)))
    ic(all_fidpairs)
    if nqubits == 1:  # special case where line-labels may be ('*',)
        if len(dataset) > 0:
            first_circuit = list(dataset.keys())[0]
            line_labels = first_circuit.line_labels
        else:
            line_labels = (0,)
        GiStr = _Circuit(idle_string, line_labels=line_labels)
    else:
        GiStr = _Circuit(idle_string, num_lines=nqubits)
    obs_infos = dict()
    errors = allerrors(nqubits, maxweight)
    fit_order = 1
    observed_error_rates = {}
    obs_error_rates_by_exp = {}
    whatever = {}
    for ifp, pauli_fidpair in all_fidpairs.items():
        all_outcomes = idt.idttools.allobservables(pauli_fidpair[1], maxweight)
        infos_for_this_fidpair = _collections.OrderedDict()
        ic(pauli_fidpair)
        for j, out in enumerate(all_outcomes):
            info = idt.compute_observed_err_rate(
                dataset,
                pauli_fidpair,
                pauli_basis_dicts,
                GiStr,
                out,
                max_lengths,
                fit_order,
            )

            infos_for_this_fidpair[out] = info
            
            obs_infos[ifp] = infos_for_this_fidpair
            observed_error_rates[ifp] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ]
            obs_error_rates_by_exp[str(pauli_fidpair[0]).replace("+",""), str(pauli_fidpair[1]).replace("+",""), str(out).replace("I","_")] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ][-1]
        whatever[pauli_fidpair] = 1
    return observed_error_rates, obs_error_rates_by_exp

In [195]:
observed_error_rates, obs_error_rates_by_exp = report_observed_rates(num_qubits, ds, max_lengths, paulidicts)

ic| all_fidpairs: {0: (State[+X], State[+X]),
                   1: (State[+X], State[+Y]),
                   2: (State[+X], State[+Z]),
                   3: (State[+Y], State[+X]),
                   4: (State[+Y], State[+Y]),
                   5: (State[+Y], State[+Z]),
                   6: (State[+Z], State[+X]),
                   7: (State[+Z], State[+Y]),
                   8: (State[+Z], State[+Z]),
                   9: (State[-X], State[+X]),
                   10: (State[-X], State[+Y]),
                   11: (State[-X], State[+Z]),
                   12: (State[-Y], State[+X]),
                   13: (State[-Y], State[+Y]),
                   14: (State[-Y], State[+Z]),
                   15: (State[-Z], State[+X]),
                   16: (State[-Z], State[+Y]),
                   17: (State[-Z], State[+Z])}
ic| pauli_fidpair: (State[+X], State[+X])
ic| obs_indices: [0]
ic| pauli_meas: State[+X]
ic| pauli_fidpair: (State[+X], State[+Y])
ic| obs_indices: [0]
ic| pauli_me

In [196]:
obs_rats = [v for v in obs_error_rates_by_exp.values()]

In [197]:
df.index

Index(['+X,+X,+X', '+X,+Y,+Y', '+X,+Z,+Z', '+Y,+X,+X', '+Y,+Y,+Y', '+Y,+Z,+Z',
       '+Z,+X,+X', '+Z,+Y,+Y', '+Z,+Z,+Z', '-X,+X,+X', '-X,+Y,+Y', '-X,+Z,+Z',
       '-Y,+X,+X', '-Y,+Y,+Y', '-Y,+Z,+Z', '-Z,+X,+X', '-Z,+Y,+Y', '-Z,+Z,+Z'],
      dtype='object')

In [198]:
obs_error_rates_by_exp

{('X', 'X', ' X'): 1.3387648131877097e-17,
 ('X', 'Y', ' Y'): 0.0,
 ('X', 'Z', ' Z'): 0.0,
 ('Y', 'X', ' X'): 0.0,
 ('Y', 'Y', ' Y'): -2.47460571227577e-07,
 ('Y', 'Z', ' Z'): 0.00020000000000000004,
 ('Z', 'X', ' X'): 0.0,
 ('Z', 'Y', ' Y'): -0.00019999999999999998,
 ('Z', 'Z', ' Z'): -2.47460571227577e-07,
 ('-X', 'X', ' X'): 2.9204839173606833e-17,
 ('-X', 'Y', ' Y'): 0.0,
 ('-X', 'Z', ' Z'): 0.0,
 ('-Y', 'X', ' X'): 0.0,
 ('-Y', 'Y', ' Y'): 2.6882662829076423e-07,
 ('-Y', 'Z', ' Z'): -0.00019999999999999998,
 ('-Z', 'X', ' X'): 0.0,
 ('-Z', 'Y', ' Y'): 0.00020000000000000004,
 ('-Z', 'Z', ' Z'): 2.6882662829076423e-07}

In [199]:
j = df.to_numpy()

In [200]:
jinv = np.linalg.pinv(j)

In [201]:
intrins_errs = jinv @ obs_rats

# Supposed intrinsic error rates (wrong)

In [202]:
dict(zip(df.columns,intrins_errs))

{'H,+X': 0.00010000000000000002,
 'H,+Y': 0.0,
 'H,+Z': 0.0,
 'S,+X': 1.2907179987760927e-07,
 'S,+Y': 1.978252065757319e-18,
 'S,+Z': 1.9737389683352295e-18,
 'C,+X,+Y': 1.3954878946477752e-21,
 'C,+X,+Z': -1.791231050312429e-24,
 'C,+Y,+Z': -3.3881317890172014e-21}